# Example Notebook for SeaDataNet Climatology data
Author(s): [Bjorn Backeberg](mailto:backeb@gmail.com) (backeb)

Creation date: 01-Aug-2019

Last updated: 04-Sept-2019

---

## Purpose

Load SeaDataNet Climatology computed from the SeaDataNet V1.1 aggregated regional datasets. Data can be downloaded [here](https://www.seadatanet.org/Products#/search?from=1&to=20).

Plot on map using cartopy

## Import necessary libraries

In [ ]:
# EGI Datahub
import os
import requests
from fs.onedatafs import OnedataFS


import xarray as xr
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

## Import dataset

### Resolve PID in DataHub

In [ ]:
# First get DataHub share from handle
PID = 'http://hdl.handle.net/21.T15999/qVk6JWQ'

r = requests.get(PID, allow_redirects=False)
share = os.path.basename(r.headers['Location'])

# And now get the path of the file in onedata
# From the share info
r = requests.get('https://datahub.egi.eu/api/v3/onezone/shares/%s' % share,
                 headers={'X-auth-token': os.environ['ONECLIENT_ACCESS_TOKEN'],
                          'Accept': 'application/json'})
space_id = r.json()['spaceId']
folder_name = r.json()['name']
# And the space info
r = requests.get('https://%s/api/v3/oneprovider/spaces/%s' % (os.environ['ONEPROVIDER_HOST'], space_id),
                 headers={'X-Auth-Token': os.environ['ONECLIENT_ACCESS_TOKEN']})
space_name = r.json()['name']
datahub_path = os.path.join('/', space_name, folder_name)

print("Data is available at %s" % datahub_path)

### Open data file at DataHub

In [ ]:
# Create connection to Oneprovider
odfs = OnedataFS(os.environ['ONEPROVIDER_HOST'],
                 os.environ['ONECLIENT_ACCESS_TOKEN'],
                 force_direct_io=True)


# load black arctic data
ds1 = xr.open_dataset(odfs.open(os.path.join(datahub_path, 'SDN_Clim_Arctic_Temperature.nc'), 'rb'))

# display some of the metadata
print(ds1)

In [ ]:
# load baltic data
ds2 = xr.open_dataset(odfs.open(os.path.join(datahub_path, 'SDN_Clim_BalticSea_Temperature.nc'), 'rb'))


In [ ]:
# and black sea data
ds3 = xr.open_dataset(odfs.open(os.path.join(datahub_path, 'SDN_Clim_BlackSea_Temperature.nc'), 'rb'))

In [ ]:
# load the data into variables for plotting
lon1 = ds1.lon.values
lat1 = ds1.lat.values
depth1 = ds1.depth.values
time1 = ds1.time.values
temperature1 = ds1.Temperature.values

lon2 = ds2.lon.values
lat2 = ds2.lat.values
depth2 = ds2.depth.values
time2 = ds2.time.values
temperature2 = ds2.Temperature.values

lon3 = ds3.lon.values
lat3 = ds3.lat.values
depth3 = ds3.depth.values
time3 = ds3.time.values
temperature3 = ds3.Temperature.values

## Plot on map using cartopy

In [ ]:
tm = 6 # set time to plot, where January=0,..,July=6

# instantiate the figure
fig = plt.figure(figsize = (13, 10), dpi = 80) 

# set cartopy projection
central_longitude = np.median(np.concatenate((lon1, lon2, lon3), axis = None)).round()
central_latitude = np.median(np.concatenate((lat1, lat2, lat3), axis = None)).round()
ax = plt.axes(projection=ccrs.NearsidePerspective(central_longitude = central_longitude,
                                                  central_latitude = central_latitude))

# define temperature colour axis bounds
bounds = [np.nanmin(np.concatenate((temperature1, temperature2, temperature3), axis = None)), 
          np.nanmax(np.concatenate((temperature1, temperature2, temperature3), axis = None))]

# plot surface data (depth = -1)
cm = ax.pcolormesh(lon1, lat1, temperature1[tm,-1,:,:], 
              shading = 'gourand', 
              cmap = plt.cm.Spectral_r, 
              vmin = bounds[0], vmax = bounds[1],
              transform = ccrs.PlateCarree())

# plot surface data (depth = -1)
cm = ax.pcolormesh(lon2, lat2, temperature2[tm,-1,:,:], 
              shading = 'gourand', 
              cmap = plt.cm.Spectral_r,
              vmin = bounds[0], vmax = bounds[1],
              transform = ccrs.PlateCarree())

# plot surface data (depth = -1)
cm = ax.pcolormesh(lon3, lat3, temperature3[tm,-1,:,:], 
              shading = 'gourand', 
              cmap = plt.cm.Spectral_r,
              vmin = bounds[0], vmax = bounds[1],
              transform = ccrs.PlateCarree())

# create the vertical temperature colobar
plt.colorbar(cm, orientation = 'vertical').set_label('[$^\circ$C]')

# Add a coastline profile in the figure.
# Possible scale values = 'intermediate', 'coarse' or 'low' 
# Depending by the scale value the computation may take time
coastline = cfeature.GSHHSFeature(scale = 'intermediate', edgecolor = 'none', facecolor = 'grey')
ax.add_feature(coastline)

# add a title
plt.title("SeaDataNet Temperature for "
          +str((pd.to_datetime(time1[tm])).strftime('%B'))
          +", depth = "+str(depth1[-1])+"m")

plt.show()